In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import sys
sys.path.append('../benchmarks')
sys.path.append('../xmen/data/')

In [4]:
from translation import Translator

In [42]:
import dataloaders
dataset = dataloaders.load_medmentions_st21pv()[0]

Found cached dataset medmentions (/home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4/cache-0e35495cd8cfaafb.arrow
Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4/cache-7960cf2ca563a297.arrow
Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4/cache-d2cb5e7ea2381bfd.arrow


In [43]:
for k in dataset.keys():
    dataset[k] = dataset[k].shuffle().select(range(0, 50))

In [44]:
t = Translator(Translator.ENGLISH, Translator.FRENCH)

[07/17/23 22:36:28] INFO     Initializing Translation Model                                       ]8;id=664662;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/notebooks/../xmen/data/translation.py\translation.py]8;;\:]8;id=136547;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/notebooks/../xmen/data/translation.py#22\22]8;;\

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
from xmen.log import shell_handler
import logging
shell_handler.setLevel(logging.INFO)

In [46]:
translated_dataset = t._translate_batch(dataset)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

[07/17/23 22:37:09] INFO     Translating documents                                               ]8;id=700027;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/notebooks/../xmen/data/translation.py\translation.py]8;;\:]8;id=384427;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/notebooks/../xmen/data/translation.py#198\198]8;;\

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [58]:
tagged = dataset.map(t._annos_to_tagged)

Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4/cache-8e10bf814bb4af8d.arrow
Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4/cache-e6d31382880ab0d9.arrow
Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___medmentions/medmentions_st21pv_bigbio_kb/1.0.0/4ed5b6a69d807969022e559198c5a7386b9a978268a558758a090db6b451d6c4/cache-dec0f4e27b03ba18.arrow


In [47]:
translated_dataset.save_to_disk('translated_50')

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

In [48]:
final_dataset = t._tags_to_entities(translated_dataset)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [49]:
len([e for v in dataset.values() for d in v for e in d['entities']])

7211

In [50]:
len([e for v in final_dataset.values() for d in v for e in d['entities']])

6484

In [72]:
import pandas as pd
def to_ent_df(ds_dict):
    df = []
    for ds in ds_dict.values():
        for d in ds:
            text = ""
            for p in d['passages']:
                for txt in p['text']:
                    text += (txt + " ")
            for e in d['entities']:
                e = e.copy()
                e['normalized'] = [n['db_id'] for n in e['normalized']]
                df.append({'document_id': d['document_id']} | e)
                for o, txt in zip(e['offsets'], e['text']):
                    assert text[o[0]:o[1]] == txt
    return pd.DataFrame(df).set_index(['document_id', 'id'])

In [73]:
df_en = to_ent_df(dataset)
df_fr = to_ent_df(final_dataset)

In [76]:
pd.merge(df_en, df_fr, left_index=True, right_index=True, suffixes=['_en', '_fr']).sample(100)

,,type_en,text_en,offsets_en,normalized_en,type_fr,text_fr,offsets_fr,normalized_fr
document_id,id,,,,,,,,
27819344,18514,T103,[cdc25C],"[[1276, 1282]]",[C0213045],T103,[cdc25C],"[[1401, 1407]]",[C0213045]
27513357,9887,T038,[inflammatory response],"[[1151, 1172]]",[C1155266],T038,[réponse inflammatoire],"[[1489, 1510]]",[C1155266]
27403338,5403,T017,[Choroidal],"[[14, 23]]",[C0008520],T017,[l'épaisseur choroïdale],"[[14, 36]]",[C0008520]
28017476,26065,T017,[Foci],"[[1234, 1238]]",[C3159064],T017,[Les foci],"[[1486, 1494]]",[C3159064]
27995352,23439,T033,[present],"[[1364, 1371]]",[C0150312],T033,[présentent],"[[1699, 1709]]",[C0150312]
...,...,...,...,...,...,...,...,...,...
28417422,38243,T168,[diet],"[[807, 811]]",[C0012155],T168,[régime],"[[899, 905]]",[C0012155]
27995352,23441,T038,[type 2 diabetes],"[[1426, 1441]]",[C0011860],T038,[diabète de type 2],"[[1769, 1786]]",[C0011860]
28347719,107931,T033,[positively],"[[752, 762]]",[C1446409],T033,[positivement],"[[843, 855]]",[C1446409]
